#Import bibliotek

In [ ]:
import sys
import numpy as np
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from skimage import io, filters, color, transform
import cv2 as cv
from scipy.ndimage.morphology import distance_transform_edt
from skimage.morphology import erosion, disk
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline

# Nazwy plików, bez rozszerzeń, wykorzystane do utworzenia zbioru danych
pages = ["dasrtseldesjdisc00frit_0007", "dasrtseldesjdisc00frit_0008", "dasrtseldesjdisc00frit_0009", "dasrtseldesjdisc00frit_0010", "dasrtseldesjdisc00frit_0011"]

#TODO: Poniżej należy podać ścieżkę do głównego folderu
folder_path = "FolderGlowny"
columns=['Char', 'Image']
sizes = range(4, 30)

In [ ]:
def remove_small_element(binary_tmp, min_size):
  #find all your connected components (white blobs in your image)
  nb_components, output, stats, centroids = cv.connectedComponentsWithStats(binary_tmp, connectivity=8)
  #connectedComponentswithStats yields every seperated component with information on each of them, such as size
  #the following part is just taking out the background which is also considered a component, but most of the time we don't want that.
  sizes = stats[1:, -1]; nb_components = nb_components - 1

  #your answer image
  binary_tmp = np.zeros((output.shape))
  #for every component in the image, you keep it only if it's above min_size
  for i in range(0, nb_components):
      if sizes[i] >= min_size:
          binary_tmp[output == i + 1] = 255
  return  binary_tmp

def find_object_in_line(bin_clip, dist):
  hist = cv.reduce(bin_clip,0, cv.REDUCE_AVG).reshape(-1)

  th = 2
  H,W = bin_clip.shape[:2]
  left = [x for x in range(W-1) if hist[x]<=th and hist[x+1]>th]
  right = [x for x in range(W-1) if hist[x]>th and hist[x+1]<=th]
  if len(left) > 0 and len(right) > 0:
    mids_vertical = [int(left[0]/2)]
    for i in range(1,len(left)):
        mid_vertical = int((left[i]+right[i-1])/2)
        #if mid_vertical > dist:
        mids_vertical.append(mid_vertical)
    mid_vertical = int(int(right[len(right)-1] + W)/2)
    mids_vertical.append(mid_vertical)
    return mids_vertical
  return []

In [ ]:
def read_single_page(page, columns):

    resultPath = folder_path+ '/TrainingFiles/AllData.csv'
    print(page)
    ##########Założenia############
    #
    #1.Wszystkie litery mają tą samą, poziomą orientację 
    #2.Na obrazie znajduje się tylko jedna kolumna wierszy 
    ###########PATH#################
    #page = "dasrtseldesjdisc00frit_0011"
    read_path = folder_path + "/" + page + ".jpg"
    text_file_path = folder_path + "/Source/" + page + ".txt"

    output_filename = resultPath
    ################Images
    image_folder_path = folder_path + "/Result/Images/"

    ###############################
    # module level variables ##########################################################################
    #Char 'a' size
    char_heigh = 40
    char_width = 15

    MIN_CONTOUR_AREA = 40
    MIN_CONTOUR_CHAR = char_heigh*char_width
    MIN_DIST_BETWEEN_WORDS = 20 #px
    DIST_BETWEN_CHARS = 1 #px
    AREA_OVER_CHAR = 0.5*char_heigh
    #Size of the digit
    #result_img_size = 50
    ###################################################################################################
    digit = io.imread(read_path)
    #print("Number of Dimension", digit.ndim)
    #io.imshow(digit)
    #io.show()
    ########################################
    if digit.ndim > 1:
        gray_image = color.rgb2gray(digit)
    else:
        gray_image = digit
    #gray_image = cv.GaussianBlur(gray_image, (5,5), 0) 
    thresh = filters.threshold_minimum(gray_image)

    binary_image = gray_image > thresh
    #io.imshow(binary_image)
    #io.show()
    binary = np.invert((binary_image*255).astype('uint8'))
    binary_final = remove_small_element(binary, MIN_CONTOUR_AREA)
    binary_final = np.invert((binary_final).astype('bool_'))
    #######################################
    dist_map = distance_transform_edt(binary_final)
    tresh_dist_map = dist_map > MIN_DIST_BETWEEN_WORDS/2

    #io.imshow(dist_map)
    #io.show()
    #io.imshow(tresh_dist_map)
    #io.show()

    neg_tresh_dist_map = np.invert(tresh_dist_map)
    mask = np.ones((MIN_DIST_BETWEEN_WORDS,1),np.uint8)
    eroded = erosion(neg_tresh_dist_map, mask)
    #io.imshow(eroded)
    #io.show()
    ####################
    eroded = (eroded*255).astype('uint8')
    ## (5) find and draw the upper and lower boundary of each lines
    hist = cv.reduce(eroded,1, cv.REDUCE_AVG).reshape(-1)

    th = 2
    H,W = digit.shape[:2]
    uppers = [y for y in range(H-1) if hist[y]<=th and hist[y+1]>th]
    lowers = [y for y in range(H-1) if hist[y]>th and hist[y+1]<=th]

    interline = [int(uppers[0]/2)]
    for i in range(1,len(uppers)):
        mid_horizontal = int((uppers[i]+lowers[i-1])/2)
        interline.append(mid_horizontal)
    mid_horizontal = int(int(lowers[len(lowers)-1] + H)/2)
    interline.append(mid_horizontal)

    digit_with_line = digit.copy()
    for y in interline:
        cv.line(digit_with_line, (0,y), (W, y), (255,0,0),6)

    lines = []
    for i in range(1, len(interline)):
        lines.append((interline[i-1], interline[i]))
    #io.imshow(digit_with_line)
    #io.show()
    #################################
    digit_with_word = digit_with_line.copy()
    complet_words = []
    for line in lines:
        line_eroded = eroded[line[0]:line[1],:]
        interword = find_object_in_line(line_eroded, MIN_DIST_BETWEEN_WORDS)
        interword = np.sort(interword)
        word = []
        for i in range(1, len(interword)):
            word.append((interword[i-1], interword[i]))
        complet_words.append((line, word))
        for x in interword:
            cv.line(digit_with_word, (x,line[0]), (x, line[1]), (255,0,0), 4)
    #io.imshow(digit_with_word)
    #io.show()  
    ##################
    digit_with_chars = digit.copy()
    text = []
    i = 0
    for line in complet_words:
        up_limit, down_limit = line[0]
        single_line = []
        i = i + 1
        #cv.putText(digit_with_chars, str(i), (10, down_limit),cv.FONT_HERSHEY_COMPLEX, 5, (0,0,255), 4, cv.LINE_AA)
        for word in line[1]: 
            word_image  = (np.invert(binary_final[up_limit:down_limit, word[0]:word[1]])*255).astype('uint8')
            interchar = find_object_in_line(word_image, DIST_BETWEN_CHARS)
            single_word = []
            for i in range(1, len(interchar)):
                char = [up_limit, down_limit, interchar[i-1]+ word[0], interchar[i]+word[0]]
                single_word.append(char)      
                
                cv.rectangle(digit_with_chars,(interchar[i-1]+ word[0],up_limit+6),(interchar[i]+word[0], down_limit-6),(0,0,255), 4)
            single_line.append(single_word)

        #for x in interchar:
        #  cv.line(digit_with_chars, (x+ word[0],up_limit), (x+word[0], down_limit), (0,255,0), 4)
        text.append(single_line)

    #io.imshow(digit_with_chars)
    #io.show()  
    ############################
    text_file = open(text_file_path, "r")
    text_in_page = text_file.read().split('\n')
    #create feature vector
    # bool_x = np.zeros((1,(result_img_size*result_img_size)), dtype='bool_')
    # bool_fv = np.zeros((1,(result_img_size*result_img_size)+1), dtype='<U32')
    from skimage import  morphology
    import pandas as pd
    table_of_data = pd.DataFrame(columns=columns)
    for num, line_coord in enumerate(text, start = 0):
        line_clip = digit_with_chars[line_coord[0][0][0]:line_coord[0][0][1],:]
        #io.imshow(line_clip)
        #io.show()
        line_text = text_in_page[num]
        text_in_page.append(line_text)
        chars = line_text.split(" ")
        i = 0
        char_sum = 0
        for word_coord in line_coord:
            char_sum = char_sum + len(word_coord)
        if len(chars) != char_sum:
            print("Line ", num, len(chars), char_sum)
        else:
            try:
                for word_coord in line_coord:
                    for char_coord in word_coord:
                        clip = binary_final[char_coord[0]:char_coord[1],char_coord[2]:char_coord[3]]
                        clip = (clip*255).astype('uint8')
                        clip = remove_small_element(clip,MIN_CONTOUR_AREA)
                        clip = (clip).astype('uint8')
                        clip = np.invert(clip)
                        clip = remove_small_element(clip,MIN_CONTOUR_AREA)
                        clip = (clip).astype('uint8')
                        #io.imshow(clip)
                        #io.show()
                        rows, = np.where(clip.sum(axis=1) > 0)
                        columns, = np.where(clip.sum(axis=0) > 0)
                        if len(rows) > 1 and len(columns) > 1 and len(rows)*len(columns) > MIN_CONTOUR_AREA:
                            tmp_img = clip[rows.min():rows.max(), columns.min():columns.max()]
                            tmp_heigh, tmp_width = tmp_img.shape
                            centre = ndi.measurements.center_of_mass(tmp_img)
                            centre = tuple((round(tup)) for tup in centre)
                            dist_row = (tmp_heigh - centre[0], centre[0])
                            dist_col = (tmp_width - centre[1], centre[1])
                            size = max(max(dist_row)*2, max(dist_col)*2)

                            back = np.zeros((size,size), dtype=type(tmp_img[0,0]))
                            row_off = round(size - max(dist_row))
                            col_off = round(size - max(dist_col))
                            
                            back[round(size/2 - centre[0]): round(size/2 +  tmp_heigh - centre[0]), 
                                round(size/2 - centre[1]): round(size/2 + tmp_width - centre[1])] = tmp_img
                            single_char_image = back
                            kernel = np.ones((3,3))
                            single_char_image = cv.morphologyEx(single_char_image.astype('uint8'), cv.MORPH_DILATE, kernel)
                            single_char_image = single_char_image/255
                            table_of_data = table_of_data.append({'Char': chars[i], 'Image':single_char_image}, ignore_index = True)
                            i= i+1
            except:
                pass
        #print("##################")
    return table_of_data

In [ ]:
allDatas = pd.DataFrame(columns=columns)
for page in pages:
    datas = read_single_page(page, columns)
    allDatas = allDatas.append(datas)

allDatas.to_csv(path_or_buf=output_filename)

dasrtseldesjdisc00frit_0007
dasrtseldesjdisc00frit_0008
dasrtseldesjdisc00frit_0009
dasrtseldesjdisc00frit_0010
Line  2 2 20
Line  15 24 3
Line  23 2 22
dasrtseldesjdisc00frit_0011


#Utworzenie zbioru nie przeskalowanych obrazów


In [ ]:
# module level variables ##########################################################################
#Size of the digit
#max numbers of one char 
max_size = 100
#enable chars
enable_chars = ['A', 'a', 'B', 'b', 'C', 'c', 'D', 'd', 'E', 'e', 'F', 'f', 'G', 'g', 'H', 'h', 'I', 'i', 'J', 'j', 'K', 'k', 'L', 'l', 'M', 'm', 'N', 'n', 'O', 'o', 'P', 'p',
                'R', 'r', 'S', 's', 'T', 't', 'U', 'u', 'W', 'w', 'Y', 'y', 'Z', 'z', 'Ä', 'ä', 'Ö', 'ö', 'Q', 'q', 'ẞ', 'ß', 'Ü', 'ü', 'V', 'v', 'X', 'x', '1','2','3', '4', 
                '5', '6', '7', '8', '9', '0', ',', '.', '(', ')', '-', '\'', '\"','+', '=','#', '^', '*', '?', '[', ']', '%', ':',';',
                'ch', 'ck', 'st', 'll', 'ss', 'fi', 'si', 'ff', 'fl']
x = {char: [] for char in enable_chars}
###################################################################################################

In [ ]:
forbidden = 0
#allDatas = allDatas.reset_index()  # make sure indexes pair with number of rows
for index, row in allDatas.iterrows():
    #print("Index:", index, "ROw:", row)
    #unzip
    _char = row['Char']
    _image = row['Image']
    try:
        x[_char].append(_image)
    #all_data = np.vstack((all_data, row[:]))

    except:
        forbidden = forbidden + 1
        print(row[0])

[]
en
ro
ib
ass
tio
ns
us
st.
be
rm
li
ts
un
kr
ns
eh
un
nn
er
un
rn
ti
em
nn
–
ass
–
–
[]
af
ts
[]
[]
[]
[]
[]
Er
[]
[]
[]
[]
ng
aß
ltu
fa
ru
Au
it
ku
fü
jü
zu
t,
Der
tu
ti
r-
li
r-
tu
en
r-
che
ru
ll-
[]
ap
tu
ft
ar
au
[]
[]
II
[]
n.
un
äf
[]
[]
[]
e-
de
de
äu
nz
Al
am
rt
ei
Ar
f,
Er
t-
af
[]


In [ ]:
false_added = 40
from skimage import util
for element in x:
    if len(x[element]) < max_size and len(x[element]) > 0:
        old_len = len(x[element])
        j = 0
        for i in range(0, false_added-old_len):
            vector = x[element][j][:]
            tmp = vector[1:]      
            tmp = np.asarray(tmp, dtype=np.float_, order='C')
            noise = np.random.normal(0, .01, tmp.shape)
            tmp = tmp + noise
            tmp = np.asarray(tmp, dtype='<U32', order='C')  
            vector[1:] = tmp
            x[element].append(vector)
            j = j + 1
            if j >= old_len:
                j = 0
    if len(x[element]) > max_size:
        x[element] = x[element][:max_size]

table_of_chars = pd.DataFrame(columns = columns)
for char in enable_chars:
    for element2  in x[char]:
        table_of_chars = table_of_chars.append({'Char': char, 'Image': element2}, ignore_index=True)


X = table_of_chars['Image']
Y = table_of_chars['Char']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y, test_size=0.2, random_state=2077)
np.savez_compressed(folder_path+ '/TrainingFiles/DataTrain(NotResized).npz',
                    X_train = X_train,X_test = X_test, Y_train = Y_train,  Y_test = Y_test)

#Funkcje umożliwiające stworzenie zbioru w docelowym rozmiarze przestrzeni cech

In [ ]:
NPZFile = np.load(folder_path+ '/TrainingFiles/DataTrain(NotResized).npz', allow_pickle= True)
X_train, Y_train, X_test, Y_test = [NPZFile[f] for f in ['X_train', 'y_train', 
                                    'X_test', 'y_test']]

In [ ]:
def make_final_data_train_file(X, result_size):
    # data_frame = pd.DataFrame(columns=['Vector'])
    result_list = []
    for x in X:
        result_size_image = cv.resize(x, dsize=(result_size, result_size), interpolation=cv.INTER_CUBIC)
        vector = result_size_image.reshape((1, (result_size*result_size)))

        result_list.append(vector[0])
        # data_frame = data_frame.append({'Vector': vector[0]}, ignore_index=True);
    return np.array(result_list)

In [ ]:
def save_to_npz_file(x_train, x_test, y_train, y_test, size):

    result_file_path = folder_path+ '/TrainingFiles/Final/' + str(size) + '.npz'
    np.savez_compressed(result_file_path,
                    x_train = x_train,x_test = x_test, y_train = y_train,  y_test = y_test)
    
def load_to_npz_file(size):
    NPZFile = np.load(folder_path+ '/TrainingFiles/Final/' + str(size) + '.npz', allow_pickle= True)
    x_train, y_train, x_test, y_test = [NPZFile[f] for f in ['x_train', 'y_train', 
                                    'x_test', 'y_test']]
    return x_train, x_test, y_train, y_test

In [ ]:
def shuffle(x_train, x_test, y_train, y_test, seed):
    x = np.vstack((x_train, x_test))
    y = np.hstack([y_train, y_test])
    new_x_train, new_x_test, new_y_train, new_y_test = train_test_split(x, y, stratify=y, test_size=0.2, random_state=seed)
    return new_x_train, new_x_test, new_y_train, new_y_test